# CCRB ChatBot Notebook

## Goal:

1. To make a tool that reads a CSV of the CCRB database
2. Adds the data to a SQL database
3. Uses an LLM to convert human questions into SQL queries
4. Uses a SQL platform like DuckDB to perform the query 
5. Uses an LLM to make the returned answer into a human readable response and presents it to the user

### Load CSV

In [61]:
import pandas as pd
import sqlalchemy as sa
import os


In [62]:
df = pd.read_csv("complaintclean.csv")

/var/folders/xv/zb6mjv_93435jvb4s_wsh3xm0000gn/T/ipykernel_12920/630838838.py:1: DtypeWarning: Columns (11,14,16,17,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("complaintclean.csv")


In [63]:
df.head()

,complaintid,shieldno,lastname,firstname,lastactivedate,daysonforce,officerrace_cat,officergender_cat,incidentranklong_cat,currentranklong_cat,...,locationtype_cat,contactreason_cat,contactoutcome_cat,impactedrace_cat,impactedgender_cat,ccrbdisposition_cat,boardcat_cat,penaltyrec_cat,subst,charges
0,202101017,4419,Ruscillo,Taylor,04/01/21,672,White,Female,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,Hispanic,Male,Complainant Unavailable,Truncated,NaN,0,NaN
1,202100294,22851,Medina,Nelson,04/01/21,4753,Hispanic,Male,Police Officer,Police Officer,...,Subway station/train,C/V intervened on behalf of/observed encounter...,Arrest - other violation/crime,Black,Male,Closed - Pending Litigation,Closed - Pending Litigation,NaN,0,NaN
2,202100294,14350,Pierre,Bebeto,04/01/21,924,Black,Male,Police Officer,Police Officer,...,Subway station/train,C/V intervened on behalf of/observed encounter...,Arrest - other violation/crime,Black,Male,Closed - Pending Litigation,Closed - Pending Litigation,NaN,0,NaN
3,202100229,29706,Chapman,John,04/01/21,361,White,Male,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,NaN,Male,Complaint Withdrawn,Truncated,NaN,0,NaN
4,202100229,13834,Tafuro,Justin,04/01/21,2733,White,Male,Police Officer,Police Officer,...,Apartment/house,Report-domestic dispute,No arrest made or summons issued,NaN,Male,Complaint Withdrawn,Truncated,NaN,0,NaN


In [64]:
df['incidentdate'].value_counts()

incidentdate
04/07/06    94
05/22/19    89
05/05/06    89
03/02/07    87
01/30/19    84
            ..
12/27/14     1
01/04/15     1
03/31/15     1
11/12/16     1
02/15/21     1
Name: count, Length: 12857, dtype: int64

### Formatting data for better compatibility with SQL

#### Starting with format dates

In [65]:
import duckdb

df_relation = duckdb.read_csv("complaintclean.csv")

# convert cols with str type dates to date type dates 

duckdb.sql("""
CREATE OR REPLACE TABLE df_relation AS
SELECT
    * REPLACE (
        TRY_STRPTIME(Incidentdate, '%m/%d/%y')::DATE AS incidentdate,
        TRY_STRPTIME(lastactivedate, '%m/%d/%y')::DATE AS lastactivedate
    )
FROM read_csv_auto('complaintclean.csv')
""")


In [66]:
db = duckdb.sql('SELECT * FROM df_relation')


In [67]:
df['incidentdate'].value_counts()

incidentdate
04/07/06    94
05/22/19    89
05/05/06    89
03/02/07    87
01/30/19    84
            ..
12/27/14     1
01/04/15     1
03/31/15     1
11/12/16     1
02/15/21     1
Name: count, Length: 12857, dtype: int64

### Adding CSV to SQL Databbase (Duck DB)

In [68]:
# Get the schema info from DuckDB
schema_info = duckdb.sql("DESCRIBE df_relation").fetchall()

# Format it into a string the AI can understand
columns = [f"{row[0]} {row[1]}" for row in schema_info]
schema_string = f"CREATE TABLE df_relation ({', '.join(columns)});"

print("Schema captured for the AI:")
print(schema_string)

Schema captured for the AI:
CREATE TABLE df_relation (complaintid BIGINT, shieldno BIGINT, lastname VARCHAR, firstname VARCHAR, lastactivedate DATE, daysonforce BIGINT, officerrace_cat VARCHAR, officergender_cat VARCHAR, incidentranklong_cat VARCHAR, currentranklong_cat VARCHAR, status_cat VARCHAR, incidentdate DATE, fadotype_cat VARCHAR, allegation_cat VARCHAR, locationtype_cat VARCHAR, contactreason_cat VARCHAR, contactoutcome_cat VARCHAR, impactedrace_cat VARCHAR, impactedgender_cat VARCHAR, ccrbdisposition_cat VARCHAR, boardcat_cat VARCHAR, penaltyrec_cat VARCHAR, subst BIGINT, charges BIGINT);


### Get User Question

In [69]:
user_question = "Which officer has the most incidents against them? Give me their full name. Ignore the badge number 0"

### Create prompt

In [70]:
prompt = f"""

You are SQL expert.

Dialect: DuckDB (PostgreSQL-like syntax)

Schema: 
{schema_string}

Rules:

- Only use columns that exist in the schema.
- Do NOT hallucinate columns.
- Return ONLY valid SQL.
- Do NOT explain anything.
- Do NOT execute anything.
- No markdown formatting.
- No backticks.

User Questions:
{user_question}
"""

In [71]:
from dotenv import load_dotenv
from portkey_ai import Portkey
load_dotenv()
portkey_api_key = os.getenv('PORTKEY_API_KEY')


### Convert Prompt to SQL

In [72]:
portkey = Portkey(
  base_url = "https://ai-gateway.apps.cloud.rt.nyu.edu/v1",
  api_key = portkey_api_key
)

response = portkey.chat.completions.create(
    model = "@openai-nyu-it-d-5b382a/gpt-4o-mini",
    messages = [
      {"role": "system", "content": "You translate natural language into SQL queries."},
      {"role": "user", "content": prompt}
    ],
    MAX_TOKENS = 512,
    temperature=0
)

print(response.choices[0].message.content)

SELECT firstname, lastname
FROM df_relation
WHERE shieldno <> 0
GROUP BY firstname, lastname
ORDER BY COUNT(complaintid) DESC
LIMIT 1;


In [73]:
sql_query = response.choices[0].message.content

sql_result = duckdb.sql(sql_query)


### Check size of SQL response to not overwhelm response LLM

In [74]:
# test = duckdb.sql("""
# SELECT * FROM df_relation 
# ORDER BY incidentdate DESC 
# LIMIT 10;

# """)
# test

In [75]:
total_rows = len(sql_result)
# renaming in case the result is too big
query_result = sql_result
clamped_result = False 

if total_rows > 20:
    query_result = duckdb.sql("SELECT * FROM query_result LIMIT 20").fetchdf()
    campled_result = True

# if clamped result pass full sql_result as an optional csv download?

#### Prompt for plain language answer

In [76]:
sql_to_text_prompt = f"""
You are a data analyst.
Your task is to translate SQL query results into clear, concise plain English.
The explanation must:
- Directly answer the user's question.
- Be fact-based and only use the data provided.
- Not invent additional statistics.
- Not speculate beyond the result.
- Not mention SQL.
- Not mention tables or schemas.
- Not include markdown formatting.
- Not include backticks.
- Be written in complete sentences.
- Use only the provided SQL result.
- Do not guess missing values.
- If the result is empty, clearly state that no records matched the criteria.
- If NULL values appear, explain that data was missing.
- If the result is a single value, respond with a direct sentence that states the value clearly.
- If multiple rows are returned, summarize key patterns, identify highest/lowest values if relevant, mention totals only if explicitly present, and keep the response under 5 sentences unless trends require more. If the result indicates additional rows beyond what is shown, acknowledge the total count when summarizing.

User question:
{user_question}
SQL query:
{sql_query}
Query result:
{query_result}
"""

In [77]:
sql_to_text_response = portkey.chat.completions.create(
    model = "@openai-nyu-it-d-5b382a/gpt-4o-mini",
    messages = [
      {"role": "system", "content": "You are a precise data analyst who explains query results in plain English. You stick strictly to the data provided, never speculate, and always write in clear, concise sentences."},
      {"role": "user", "content": sql_to_text_prompt}
    ],
    MAX_TOKENS = 512,
    temperature=0
)


In [78]:
print(sql_to_text_response.choices[0].message.content)

The officer with the most incidents against them is Robert Rodriguez.
